In [21]:
!pip install --upgrade pip
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-d870y8eo
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-d870y8eo
  Resolved https://github.com/huggingface/transformers.git to commit 608e163b527eaee41e650ffb9eb4c422d2679902
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [23]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb') as f:
    f.write(b)
  return 'audio.wav'  # or webm ?

In [41]:
# Record
duration = 35 # seconds
myrecording = record(duration)

<IPython.core.display.Javascript object>

In [25]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline,AutoModelForSequenceClassification,AutoTokenizer,AutoFeatureExtractor
import numpy
import torch

In [26]:
model_name2 = "openai/whisper-large-v3" #change to whisper small to process the audio faster
speechmodel = AutoModelForSpeechSeq2Seq.from_pretrained(model_name2)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

In [37]:
pipe = pipeline("automatic-speech-recognition", model=speechmodel, tokenizer=AutoTokenizer.from_pretrained(model_name2), feature_extractor = AutoFeatureExtractor.from_pretrained(model_name2), device=0 if torch.cuda.is_available() else -1)
# myrecording = numpy.array(myrecording)
predicted_ids1 = pipe(myrecording,max_new_tokens=100) #make it to chunk so whisper can handle bigger file
#predicted_ids2 = pipe.generate("hello", forced_decoder_ids=processor.tokenizer.get_decoder_prompt_ids(language='en', task='translate'))
transcription = predicted_ids1['text']
#translation = processor.batch_decode(predicted_ids2, skip_special_tokens=True)[0]
transcription


Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


ValueError: You have passed more than 3000 mel input features (> 30 seconds) which automatically enables long-form generation which requires the model to predict timestamp tokens. Please either pass `return_timestamps=True` or make sure to pass no more than 3000 mel input features.

In [42]:
pipe = pipeline("automatic-speech-recognition", model=speechmodel, tokenizer=AutoTokenizer.from_pretrained(model_name2), feature_extractor = AutoFeatureExtractor.from_pretrained(model_name2), device=0 if torch.cuda.is_available() else -1)
predicted_ids1 = pipe(myrecording,max_new_tokens=100, chunk_length_s=10) #make it to chunk so whisper can handle bigger file
transcription = predicted_ids1['text']
transcription

Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


" Highly intelligent dogs have the capability of expressing their joy and happiness by wagging their tails. They are known to be the most loyal of animals. Dogs are sense your pain and can be your best friend. They can sense emotions and when you're sad, they become sad and when you're happy, they are happy. Okay. Bye."